### Imports

In [1]:
import os
from os import path
from os.path import join, relpath
import random

import xml.etree.ElementTree as ET

# from tflite_model_maker import ExportFormat
# from tflite_model_maker import model_spec
# from tflite_model_maker import object_detector

# import tensorflow as tf
# tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

%load_ext autoreload
%autoreload 2

### Helpful constants

In [4]:
colors = 'red','green','purple'
nums = 'one','two','three'
shades = 'empty','partial','full'
shapes = 'diamond','oval','squiggle'
label_names = [f'{num} {color} {shape} {shade}' for num in nums for color in colors for shape in shapes for shade in shades]

### Data formatting

In [3]:
def get_set(path):
    data = pd.DataFrame(columns=['filepath', 'class_color', 'class_num', 'class_shape', 'class_shade'])
    for dirpath, subdirs, files in os.walk(path):
        for file in files:
            filepath = relpath(join(dirpath, file))
            strpath = str(filepath).split('\\')
            class_num = strpath[-5]
            class_color = strpath[-4]
            class_shape = strpath[-3]
            class_shade = strpath[-2]
            row = pd.DataFrame({'filepath': filepath,
                             'class_color': class_color,
                             'class_num': class_num,
                             'class_shape': class_shape,
                             'class_shade': class_shade},
                               columns=['filepath', 'class_color', 'class_num', 'class_shape', 'class_shade'],
                               index=[0])
            data = pd.concat([data, row], ignore_index=True)
    return data

data_panda = get_set('./dataSET')
display(data_panda.head(50))

,filepath,class_color,class_num,class_shape,class_shade
0,dataSET\one\green\diamond\empty\101.png,green,one,diamond,empty
1,dataSET\one\green\diamond\empty\120.png,green,one,diamond,empty
2,dataSET\one\green\diamond\empty\172.png,green,one,diamond,empty
3,dataSET\one\green\diamond\empty\189.png,green,one,diamond,empty
4,dataSET\one\green\diamond\empty\2.png,green,one,diamond,empty
5,dataSET\one\green\diamond\empty\205.png,green,one,diamond,empty
6,dataSET\one\green\diamond\empty\222.png,green,one,diamond,empty
7,dataSET\one\green\diamond\empty\254.png,green,one,diamond,empty
8,dataSET\one\green\diamond\empty\286.png,green,one,diamond,empty
9,dataSET\one\green\diamond\empty\31.png,green,one,diamond,empty


In [5]:
data = get_set('./dataSET')

mlb = MultiLabelBinarizer(classes=colors + nums + shades + shapes)
labels = mlb.fit_transform(data[data.columns[1:]].values)
label_frame = pd.DataFrame({key : row for key, row in zip(mlb.classes, labels.T)})
binarized_data = pd.concat((data['filepath'], label_frame), axis=1)

display(binarized_data.head(50))
binarized_data.to_csv('labels.csv', index=False)

,filepath,red,green,purple,one,two,three,empty,partial,full,diamond,oval,squiggle
0,dataSET\one\green\diamond\empty\101.png,0,1,0,1,0,0,1,0,0,1,0,0
1,dataSET\one\green\diamond\empty\120.png,0,1,0,1,0,0,1,0,0,1,0,0
2,dataSET\one\green\diamond\empty\172.png,0,1,0,1,0,0,1,0,0,1,0,0
3,dataSET\one\green\diamond\empty\189.png,0,1,0,1,0,0,1,0,0,1,0,0
4,dataSET\one\green\diamond\empty\2.png,0,1,0,1,0,0,1,0,0,1,0,0
5,dataSET\one\green\diamond\empty\205.png,0,1,0,1,0,0,1,0,0,1,0,0
6,dataSET\one\green\diamond\empty\222.png,0,1,0,1,0,0,1,0,0,1,0,0
7,dataSET\one\green\diamond\empty\254.png,0,1,0,1,0,0,1,0,0,1,0,0
8,dataSET\one\green\diamond\empty\286.png,0,1,0,1,0,0,1,0,0,1,0,0
9,dataSET\one\green\diamond\empty\31.png,0,1,0,1,0,0,1,0,0,1,0,0


### Image Format
TensorFlow Lite Model Maker requires JPEG images

In [6]:
from PIL import Image
os.makedirs('dataSET.jpg', exist_ok=True)
for dirpath, dirname, filenames in os.walk('dataSET'):
    for file in filenames:
        png = Image.open(join(dirpath, file))
        png.save(join('dataSET.jpg', f'{"_".join(dirpath.split(os.sep)[1:])}_{path.splitext(file)[0]}.jpg'))

### XML data
During label process, png filenames were saved to XML annotations. The code below changes them to the new jpg file extensions.

In [7]:
for dirpath, dirname, filenames in os.walk('boxes'):
    for file in filenames:
        tree = ET.parse(join(dirpath, file))
        root = tree.getroot()
        
        filename_tag = root.find('filename')
        filename_tag.text = f'{"_".join(dirpath.split(os.sep)[1:])}_{path.splitext(file)[0]}.jpg'
        
        folder_tag = root.find('folder')
        folder_tag.text = join(*dirpath.split(os.sep)[1:])
        tree.write(join(dirpath, file))

In [6]:
def split():
    annotations = [relpath(join(dirpath, path.splitext(file)[0]), 'boxes') for (dirpath, dirnames, filenames) in os.walk('boxes') for file in filenames]
    random.shuffle(annotations)
    train_annotations = []
    validation_annotations = []
    test_annotations = []
    for i in range(len(annotations)):
        if i < .8 * len(annotations):
            train_annotations.append(annotations[i])
        elif i < .9 * len(annotations):
            validation_annotations.append(annotations[i])
        else:
            test_annotations.append(annotations[i])

    train_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=train_annotations, label_map=label_names)
    validation_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=validation_annotations, label_map=label_names)
    test_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=test_annotations, label_map=label_names)
    
    return train_data, validation_data, test_data

In [9]:
train_data, val_data, test_data = split()

In [18]:
low_latency_spec = model_spec.get('efficientdet_lite0')
high_precision_spec = model_spec.get('efficientdet_lite4')

In [ ]:
low_latency_model = object_detector.create(train_data, model_spec=low_latency_spec, validation_data=val_data)
low_latency_model.export(tflite_filename='ll_model.tflite', export_dir='../assets')

In [28]:
high_precision_model = object_detector.create(train_data, model_spec=low_latency_spec, validation_data=val_data)
high_precision_model.export(tflite_filename='hp_model.tflite', export_dir='../assets')

Epoch 1/50
22/22 [==============================] - 145s 6s/step - det_loss: 2.2874 - cls_loss: 1.7767 - box_loss: 0.0102 - reg_l2_loss: 0.0184 - loss: 2.3059 - learning_rate: 0.0439 - gradient_norm: 0.4848 - val_det_loss: 2.2706 - val_cls_loss: 1.8240 - val_box_loss: 0.0089 - val_reg_l2_loss: 0.0184 - val_loss: 2.2891
Epoch 2/50
22/22 [==============================] - 131s 6s/step - det_loss: 1.9870 - cls_loss: 1.7232 - box_loss: 0.0053 - reg_l2_loss: 0.0185 - loss: 2.0055 - learning_rate: 0.0798 - gradient_norm: 0.5031 - val_det_loss: 2.1309 - val_cls_loss: 1.7691 - val_box_loss: 0.0072 - val_reg_l2_loss: 0.0185 - val_loss: 2.1493
Epoch 3/50
22/22 [==============================] - 132s 6s/step - det_loss: 1.8110 - cls_loss: 1.6508 - box_loss: 0.0032 - reg_l2_loss: 0.0185 - loss: 1.8295 - learning_rate: 0.0795 - gradient_norm: 0.6616 - val_det_loss: 1.9400 - val_cls_loss: 1.6852 - val_box_loss: 0.0051 - val_reg_l2_loss: 0.0185 - val_loss: 1.9586
Epoch 4/50
22/22 [===================

## Results

In [ ]:
label_map = {num: label for num, label in zip(range(1, len(label_names) + 1), label_names)}

In [30]:
ll_obj_detector = object_detector.ObjectDetector(model_spec=low_latency_spec, label_map=test_data.label_map)
ll_obj_detector.evaluate_tflite('ll_model.tflite', test_data)

  3/176 [..............................] - ETA: 12:39

KeyboardInterrupt: 

In [32]:
hh_obj_detector = object_detector.ObjectDetector(model_spec=high_precision_spec, label_map=test_data.label_map)
hh_obj_detector.evaluate_tflite('hp_model.tflite', test_data)

AttributeError: 'PrefetchDataset' object has no attribute 'gen_dataset'

### Running Model

In [23]:
from imageio import imread

In [27]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="ll_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.asarray(imread('dataSET/one/green/diamond/empty/2.png'))
interpreter.set_tensor(input_details[0]['index'], [input_data])

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

ValueError: Cannot set tensor: Dimension mismatch. Got 3 but expected 320 for dimension 1 of input 0.

# MediaPipe Model Maker

In [1]:
import json
import tensorflow as tf
print(tf.config.list_physical_devices())
from mediapipe_model_maker import object_detector
import shutil

import os
from os import path
from os.path import join, relpath
import random

import xml.etree.ElementTree as ET

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

import numpy as np

2024-03-20 11:41:21.009213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


/home/sdubs/.conda/envs/set-solver/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Pre processing
Creates pascal voc folder in format: Folder structure should be:
```
<data_dir>/
    images/
        <file0>.jpg ...
    Annotations/
        <file0>.xml ...
```

Each <file0>.xml annotation file should have the following format:
```
<annotation>
    <filename>file0.jpg</filename> <object>

    <name>kangaroo</name> 
    <bndbox>
        <xmin>233</xmin> <ymin>89</ymin> <xmax>386</xmax> <ymax>262</ymax>
    </bndbox>
    </object> <object>...</object>
</annotation>
```

In [2]:
for dirpath, dirname, filenames in os.walk('boxes'):
    for file in filenames:
        tree = ET.parse(join(dirpath, file))
        filename = f'{"_".join(dirpath.split(os.sep)[1:])}_{path.splitext(file)[0]}.xml'
        tree.write(join('pascal_voc_data/boxes', filename))

In [3]:
def split(train_dir, validation_dir):
    files = os.listdir('pascal_voc_data/boxes')
    indices = np.random.choice(np.arange(len(files)), size=int(.2*len(files)), replace=False)
    
    # clean previous data directories
    for dir in [train_dir, validation_dir]:
        shutil.rmtree(dir)
        os.makedirs(join(dir, 'images'))
        os.makedirs(join(dir, 'Annotations'))
    
    print('copying files', len(files))
    for i, file in enumerate(files):
        filename = path.splitext(file)[0]

        if i in indices:
            dir = validation_dir
        else:
            dir = train_dir

        print(i, dir, filename)
        shutil.copy2(join('pascal_voc_data/dataSET.jpg', f'{filename}.jpg'), join(dir, 'images'))
        shutil.copy2(join('pascal_voc_data/boxes', f'{filename}.xml'), join(dir, 'Annotations'))

def train(train_dir, validation_dir):
    print('training')
    train_data = object_detector.Dataset.from_pascal_voc_folder(data_dir=train_dir, max_num_images=1)
    print('done', train_data)
    validation_data = object_detector.Dataset.from_pascal_voc_folder(data_dir=validation_dir, max_num_images=1)
    # print('validation done', validation_data)
    # # test_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=test_annotations, label_map=label_names)

    return train_data, validation_data

In [4]:
train_dir = 'pascal_voc_data/train'
validation_dir = 'pascal_voc_data/validation'

In [5]:
# split(train_dir, validation_dir)

In [ ]:
# train, validation = train(train_dir, validation_dir)
# print(train, validation)
train_data = object_detector.Dataset.from_pascal_voc_folder(data_dir=train_dir)
validation_data = object_detector.Dataset.from_pascal_voc_folder(data_dir=validation_dir)
# print('validation done', validation_data)
# # test_data = object_detector.DataLoader.from_pascal_voc(images_dir='dataSET.jpg', annotations_dir='boxes', annotation_filenames=test_annotations, label_map=label_names)

print(train_data.size, validation_data.size)

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)


## Evaluation

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


## Export

In [ ]:
model.export_model(f'{spec}-model.tflite')

## Pytorch

In [6]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device = 'cpu'
# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# ``FasterRCNN`` needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# ``OrderedDict[Tensor]``, and in ``featmap_names`` you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7,
    sampling_ratio=2
)

# put the pieces together inside a Faster-RCNN model
model = FasterRCNN(
    backbone,
    num_classes=75,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler).to(device)

In [3]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomVerticalFlip(0.5))
        transforms.append(T.ColorJitter(brightness=1))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [3]:
import utils
from dataSET import DataSET
from torch.utils.data import DataLoader
dataset = DataSET('pascal_voc_data', None, device=device)
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

# For Training
images, targets = next(iter(data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print('output', output)

# For inference
# model.eval()
# x = [torch.rand(3, 300, 400).to(device), torch.rand(3, 500, 400).to(device)]
# predictions = model(x)  # Returns predictions
# print(predictions)

NameError: name 'device' is not defined

In [34]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [22]:
dataset = DataSET('pascal_voc_data', None, device=device)
labels = set()
for image, data in dataset:
    labels.add(data['labels'][0].item())

print(labels)
print(len(labels))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81}
74


In [11]:
import os
from os import path
from os.path import join, relpath
import random

import xml.etree.ElementTree as ET
import shutil

def pascal2ultra(source, dest):
    for src_mode, dest_mode in zip(['train', 'validation'], ['train', 'val']):
        if not path.exists(join(dest, 'images', dest_mode)):
            shutil.copytree(join(source, src_mode, 'images'), join(dest, 'images', dest_mode))
        
        os.makedirs(join(dest, 'labels', dest_mode), exist_ok=True)
        for annotation in os.listdir(join(source, src_mode, 'Annotations')):
            filename = join(source, src_mode, 'Annotations', annotation)

            if not path.exists(join(dest, 'labels', dest_mode, f'${filename}.txt')):
                tree = ET.parse(filename)
                root = tree.getroot()
        
                image_width = int(root.find('size/width').text)
                image_height = int(root.find('size/height').text)
        
                lines = []
                objects = root.findall('object')
                for tag in objects:
                    label = label_names.index(tag.find('name').text)
                    box = tag.find('bndbox')
        
                    xmin = int(box.find('xmin').text)
                    xmax = int(box.find('xmax').text)
                    ymin = int(box.find('ymin').text)
                    ymax = int(box.find('ymax').text)
        
                    width = (xmax - xmin) / image_width
                    height = (ymax - ymin) / image_height
                    x_center = (xmax + xmin) / 2 / image_width
                    y_center = (ymax + ymin) / 2 / image_height
                    lines.append(" ".join([str(label), str(x_center), str(y_center), str(width), str(height)]))
                
            with open(join(dest, 'labels', dest_mode, f'{path.splitext(path.basename(filename))[0]}.txt'), 'w') as f:
                f.writelines(lines)
            
pascal2ultra('pascal_voc_data', 'datasets/dataSET')

In [2]:
import torch
import os
from ultralytics import YOLO
from torch.utils.mobile_optimizer import optimize_for_mobile

model = YOLO('runs/detect/train2/weights/best.pt')
os.makedirs('models', exist_ok=True)

model.export(format='torchscript', optimize=True, int8=True)
model.export(format='tflite', int8=True)
# model_q = torch.quantization.convert(model)
# 
# scripted_model = model.export(format='torchscript')
# print(scripted_model)
# scripted_model_q = model_q.export(format='torchscript')
# print(scripted_model_q)
# 
# opt_model = optimize_for_mobile(scripted_model)
# opt_model_q = optimize_for_mobile(scripted_model_q)
# 
# opt_model._save_for_lite_interpreter('models/opt_model.ptl')
# opt_model_q._save_for_lite_interpreter('models/opt_mode_q.ptl')

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.3.1+rocm6.0 CPU (AMD Ryzen 5 3600 6-Core Processor)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
Model summary (fused): 168 layers, 3160775 parameters, 0 gradients, 8.8 GFLOPs

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 256, 256) BCHW and output shape(s) (1, 85, 1344) (6.2 MB)

TorchScript: starting export with torch 2.3.1+rocm6.0...
TorchScript: optimizing for mobile...
TorchScript: export success ✅ 1.2s, saved as 'runs/detect/train2/weights/best.torchscript' (12.1 MB)

Export complete (2.6s)
Results saved to /run/media/sdubs/Windows/Users/swrig/IdeaProjects/set_solver/training_model/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.torchscript imgsz=256 int8 
Validate:        yolo val task=detect model=runs/detect/train2/weights/best.torchscript imgsz=256 data=dataSET.yaml int8 
Vis

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe-model-maker 0.2.1.3 requires tensorflow>=2.10, which is not installed.
tensorflow-text 2.14.0 requires tensorflow<2.15,>=2.14.0; platform_machine != "arm64" or platform_system != "Darwin", which is not installed.
tf-models-official 2.14.2 requires tensorflow~=2.14.0, which is not installed.
tensorflow-metadata 1.14.0 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 282.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 243.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 46.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.8/200.8 MB 44.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 226.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 261.5 MB/s eta 0:00:00
  Attempting unins

100%|██████████| 1.11M/1.11M [00:00<00:00, 36.7MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /run/media/sdubs/Windows/Users/swrig/IdeaProjects/set_solver/training_model/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 23.81file/s]


ONNX: starting export with onnx 1.16.1 opset 17...


ONNX: slimming with onnxslim 0.1.31...
ONNX: export success ✅ 0.9s, saved as 'runs/detect/train2/weights/best.onnx' (12.2 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=coco8.yaml'


Scanning /run/media/sdubs/Windows/Users/swrig/IdeaProjects/set_solver/training_model/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

TensorFlow SavedModel: WARNING ⚠️ >300 images recommended for INT8 calibration, found 4 images.
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...

Automatic generation of each OP name started ========================================


Automatic generation of each OP name complete!

Model loaded ========================================================================

Model conversion started ============================================================
saved_model output started ==========================================================
saved_model output complete!
Float32 tflite output complete!
Float16 tflite output complete!
Dynamic Range Quantization tflite output complete!
Input signature information for quantization
signature_name: serving_default
input_name.0: images shape: (1, 256, 256, 3) dtype: <dtype: 'float32'>
INT8 Quantization tflite output complete!
Full INT8 Quantization tflite output complete!
INT8 Quantization with int16 activations tflite output complete!
Full INT8 Quantization with int16 activations tflite output complete!
TensorFlow SavedModel: export success ✅ 69.1s, saved as 'runs/detect/train2/weights/best_saved_model' (40.3 MB)

TensorFlow Lite: starting export with tensorflow 2.14.0.600...
T

'runs/detect/train2/weights/best_saved_model/best_int8.tflite'

In [3]:
script_model = YOLO('runs/detect/train2/weights/best.torchscript')
script_model._save_for_lite_interpreter('models/opt_model.ptl')

AttributeError: 'YOLO' object has no attribute '_save_for_lite_interpreter'